# Лабораторная работа № 1

## Данные

In [ ]:
!rm -rf '/content/benchmarks/benchmarks'

In [ ]:
!wget https://www.dropbox.com/s/i6lcluipzhi2rfu/benchmarks.rar?dl=0 -O benchmarks.rar
!unrar x "/content/benchmarks.rar"

--2022-05-16 06:15:06--  https://www.dropbox.com/s/i6lcluipzhi2rfu/benchmarks.rar?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.85.18, 2620:100:6017:18::a27d:212
Connecting to www.dropbox.com (www.dropbox.com)|162.125.85.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/i6lcluipzhi2rfu/benchmarks.rar [following]
--2022-05-16 06:15:06--  https://www.dropbox.com/s/raw/i6lcluipzhi2rfu/benchmarks.rar
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucb28406e3a8aaa9aefa83ee57f5.dl.dropboxusercontent.com/cd/0/inline/BlW43dLAoT8xhQ0766QYOZafVad3qQL1TgNT3et9_6Vyir9Rs2W4EQpTDDOfMGHGyob0k1bePzBQj6DALtn-2q5J2CNo4EywENCUkMJ6MshoFfuTcvWHPfDRDDSS1NnJzwP93LdFcW9dpGPgmdc0NJPNJrN2sIArpPoLaTUSi_diKA/file# [following]
--2022-05-16 06:15:07--  https://ucb28406e3a8aaa9aefa83ee57f5.dl.dropboxusercontent.com/cd/0/inline/BlW43dLAoT8xhQ0766QYOZafVad3qQL1TgNT3et9_6Vyir9Rs2W4EQp

In [ ]:
from os import listdir
from os.path import isfile, join
folder = '/content/benchmarks'
files = sorted([f for f in listdir(folder) if isfile(join(folder, f))])
good_tests = {'t': [], 'w': []}
bad_tests = {'t': [], 'w': []}
for filename in files:
  with open(join(folder, filename), 'r') as f:
    content = f.read()
    if 'good' in filename:
      if 'w' in filename:
        good_tests['w'].append(content)
      else:
        good_tests['t'].append(content)
    else:
      if 'w' in filename:
        bad_tests['w'].append(content)
      else:
        bad_tests['t'].append(content)
good_tests

{'t': ['Гостиная Анны Павловны начала понемногу наполняться. Приехала высшая знать Петербурга, люди самые разнородные по возрастам и характерам, но одинаковые по обществу, в каком все жили; приехала дочь князя Василия, красавица Элен, заехавшая за отцом, чтобы с ним вместе ехать на праздник посланника. Она была в шифре и бальном платье. Приехала и известная, как la femme la plus séduisante de Pétersbourg 1, молодая, маленькая княгиня Болконская, прошлую зиму вышедшая замуж и теперь не выезжавшая в большой свет по причине своей беременности, но ездившая еще на небольшие вечера. Приехал князь Ипполит, сын князя Василия, с Мортемаром, которого он представил; приехал и аббат Морио и многие другие.',
  "Маленькая княгиня, переваливаясь, маленькими быстрыми шажками обошла стол с рабочею сумочкой на руке и, весело оправляя платье, села на диван, около серебряного самовара, как будто все, что она ни делала, было partie de plaisir 3 для нее и для всех ее окружавших.\n— J'ai apporté mon ouvrage 

In [ ]:
def test_func(func):
  for test_num in range(len(bad_tests['t'])):
    assert func(bad_tests['t'][test_num], bad_tests['w'][test_num]) == bad_tests['t'][test_num].find(bad_tests['w'][test_num])

## Наивный алгоритм 

In [ ]:
def naive_string_matcher(text, pattern, counter=None):
  is_found = False
  for i in range(0, len(text) - len(pattern) + 1):
    for j in range(i, i + len(pattern)):
      if text[j] == pattern[j - i]:
        if counter:
          counter[0] +=1
        is_found = True
      else: 
        if counter:
          counter[0] +=1
        is_found = False
        break
    if is_found:
      return i
  return -1

## Алгоритм Бойера-Мура-Хорспула

In [ ]:
def get_shift_table(pattern, counter=None):
  shifts = {}
  ind = 0
  for i in range(len(pattern) - 2, -1, -1):
    ch = pattern[i]
    if counter:
      counter[0] += 1
    ind += 1
    if ch not in shifts:
      shifts[ch] = ind
  if pattern[-1] not in shifts:
    shifts[pattern[-1]] = len(pattern)
  return shifts

In [ ]:
def BoyerMoorHorspool(text, pattern, counter=None):
  i = len(pattern) - 1
  j = len(pattern) - 1

  shifts = get_shift_table(pattern, counter)
  while(i < len(text)):
    if counter:
      counter[0] += 1
    if text[i] != pattern[j]:
      if j == len(pattern) - 1:
        i += shifts[text[i]] if text[i] in shifts else len(pattern)
      else:
        i += shifts[pattern[-1]] + (len(pattern) - 1 - j)
        j = len(pattern) - 1
    else:
      j -= 1
      i -= 1
      if j == -1:
        return i + 1
  return -1


## Алгоритм Кнута-Мориса-Пратта

In [ ]:
def prefix_func(pattern, counter=None):
  pi = [0] * len(pattern)
  i = 1
  j = 0
  while i != len(pattern):
    if counter:
      counter[0] += 1
    if pattern[i] == pattern[j]:
      j += 1
      pi[i] =j 
      i += 1
    else:
      if j != 0:
        j = pi[j - 1]
      else:
        pi[i] = 0
        i += 1
  return pi

In [ ]:
prefix_func('aabaaab')

[0, 1, 0, 1, 2, 2, 3]

In [ ]:
def KnuthMorrisPratt(text, pattern, counter=None):
  i = 0
  j = 0

  pi = prefix_func(pattern, counter)
  while i != len(text):
    if counter:
      counter[0] += 1
    if text[i] == pattern[j]:
      i += 1
      j += 1
      if j == len(pattern):
        return i - j
    else:
      if j == 0:
        i += 1
        if i == len(text):
          return -1
      else:
        j = pi[j - 1]

## Тесты

In [ ]:
from time import time

In [ ]:
def benchmark_func(func):
  avg_time = {'bad_tests': 0, 'good_tests': 0}
  for _ in range(100):
    avg_iter_time = 0
    for test_num in range(len(bad_tests['t'])):
      start_time = time()
      func(bad_tests['t'][test_num], bad_tests['w'][test_num])
      avg_iter_time += time() - start_time
    avg_time['bad_tests'] += avg_iter_time / len(bad_tests['t'])
    avg_iter_time = 0
    for test_num in range(len(good_tests['t'])):
      start_time = time()
      func(good_tests['t'][test_num], good_tests['w'][test_num])
      avg_iter_time += time() - start_time
    avg_time['good_tests'] += avg_iter_time / len(good_tests['t'])

  avg_time['bad_tests'] /= 100
  avg_time['good_tests'] /= 100
  return avg_time['bad_tests'], avg_time['good_tests']

### Среднее время работы алгоритмов:

In [ ]:
import pandas as pd

In [ ]:
functions = {
    'Naive': naive_string_matcher,
    'Boyer-Moor-Horspool': BoyerMoorHorspool,
    'Knuth-Morris-Pratt': KnuthMorrisPratt
}

results = {}
for name, func in functions.items():
  results[name] = benchmark_func(func)
pd.DataFrame.from_dict(results, columns=['bad tests', 'good tests'], orient='index')

,bad tests,good tests
Naive,0.130772,0.001548
Boyer-Moor-Horspool,0.000674,0.000146
Knuth-Morris-Pratt,0.000920,0.001235


### Время на каждый тест:

In [ ]:
def get_time_test(func):
  current_bad_time = []
  current_good_time = []

  for test_num in range(len(bad_tests['t'])):
    avg_time = 0
    for _ in range(100):
      start_time = time()
      func(bad_tests['t'][test_num], bad_tests['w'][test_num])
      avg_time += time() - start_time
    current_bad_time.append(avg_time / 100)


  for test_num in range(len(good_tests['t'])):
    avg_time = 0
    for _ in range(100):
      start_time = time()
      func(good_tests['t'][test_num], good_tests['w'][test_num])
      avg_time += time() - start_time
    current_good_time.append(avg_time / 100)
    
  return current_bad_time + current_good_time

In [ ]:
columns=['bad_1', 'bad_2', 'bad_3', 'bad_4', 'good_1', 'good_2', 'good_3', 'good_4']
results = {}
for name, func in functions.items():
  results[name] = get_time_test(func)
pd.DataFrame.from_dict(results, orient='index', columns=columns)

,bad_1,bad_2,bad_3,bad_4,good_1,good_2,good_3,good_4
Naive,0.000011,0.000139,0.010798,0.504784,0.000272,0.000292,0.000968,0.004709
Boyer-Moor-Horspool,0.000005,0.000040,0.000415,0.002249,0.000039,0.000049,0.000204,0.000251
Knuth-Morris-Pratt,0.000005,0.000058,0.000594,0.003084,0.000219,0.000281,0.000851,0.003528


### Количество операций на каждый тест:

In [ ]:
def get_operation_amount(func):
  counter_all = []

  for test_num in range(len(bad_tests['t'])):
    counter = [0]
    func(bad_tests['t'][test_num], bad_tests['w'][test_num], counter)
    counter_all.append(counter[0])

  for test_num in range(len(good_tests['t'])):
    counter = [0]
    func(good_tests['t'][test_num], good_tests['w'][test_num], counter)
    counter_all.append(counter[0])

  return counter_all

In [ ]:
results = {}
for name, func in functions.items():
  results[name] = get_operation_amount(func)
pd.DataFrame.from_dict(results, orient='index', columns=columns)

,bad_1,bad_2,bad_3,bad_4,good_1,good_2,good_3,good_4
Naive,18,910,90100,4001000,633,695,2074,9614
Boyer-Moor-Horspool,11,109,1099,5999,96,191,814,601
Knuth-Morris-Pratt,19,207,2097,10997,650,779,2459,9705
